<a href="https://colab.research.google.com/github/HimashiRathnayake/Hate-Speech-Humor-Detection/blob/branch-1/Annotation/Inter_Annotation_Agreement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**User Parameters**

In [5]:
tags_no = 2
sentences_no = 10
annotators = ['Raveesha','Himashi']

**Read UIMA CAS XMI File to get data**

In [6]:
!pip install dkpro-cassis > /dev/null

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from cassis import *
import numpy as np

matrix_to_kappa = np.zeros((sentences_no, tags_no), dtype=int)

for annotator in annotators:

  # Read humor tags from Himashi
  with open('/content/drive/Shareddrives/FYP-CodeStars/Annotations/'+annotator+'/TypeSystem.xml', 'rb') as f:
    typesystem = load_typesystem(f)

  with open('/content/drive/Shareddrives/FYP-CodeStars/Annotations/'+annotator+'/Kappa_Annotation_'+annotator+'.xmi', 'rb') as f:
    doc = load_cas_from_xmi(f, typesystem=typesystem)

  with open('sentiment-sentence-per-line.txt', 'w') as f:
    for (index, sentence) in enumerate(doc.select('webanno.custom.Humor')):
      if (sentence.Humor=="Humorous"):
        matrix_to_kappa[index,0]=matrix_to_kappa[index,0]+1
      elif (sentence.Humor=="Non-humorous"):
        matrix_to_kappa[index,1]=matrix_to_kappa[index,1]+1

print(matrix_to_kappa)

[[0 2]
 [0 2]
 [0 2]
 [0 2]
 [0 2]
 [0 2]
 [0 2]
 [0 2]
 [0 2]
 [0 2]]


**Calculate Cohen Kappa**

In [9]:
# import sklearn.metrics
# sklearn.metrics.cohen_kappa_score(['humorous','non-humorous','humorous'], ['humorous','non-humorous',''], labels=None, weights=None, sample_weight=None)

**Calculate Fleiss Kappa**

In [10]:
def fleiss_kappa(M):
    """Computes Fleiss' kappa for group of annotators.
    :param M: a matrix of shape (:attr:'N', :attr:'k') with 'N' = number of subjects and 'k' = the number of categories.
        'M[i, j]' represent the number of raters who assigned the 'i'th subject to the 'j'th category.
    :type: numpy matrix
    :rtype: float
    :return: Fleiss' kappa score
    """
    N, k = M.shape  # N is # of items, k is # of categories
    n_annotators = float(np.sum(M[0, :]))  # # of annotators
    tot_annotations = N * n_annotators  # the total # of annotations
    category_sum = np.sum(M, axis=0)  # the sum of each category over all items

    # chance agreement
    p = category_sum / tot_annotations  # the distribution of each category over all annotations
    PbarE = np.sum(p * p)  # average chance agreement over all categories

    # observed agreement
    P = (np.sum(M * M, axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))
    Pbar = np.sum(P) / N  # add all observed agreement chances per item and divide by amount of items

    return round((Pbar - PbarE) / (1 - PbarE), 4)

fleiss_kappa(matrix_to_kappa)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


nan

In [12]:
import statsmodels
from statsmodels.stats.inter_rater import fleiss_kappa
fleiss_kappa(np.array(matrix_to_kappa))

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/inter_rater.py:266: RuntimeWarning: invalid value encountered in double_scalars
  kappa = (p_mean - p_mean_exp) / (1- p_mean_exp)


nan